# Radyal Temelli Fonksiyon Ağları (Radial Basis Function Networks) (RBFNs)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

**Sınıflandırma Problemi:** İris veri seti 3 farklı sınıfa ait sınıflandırma problemi   

In [2]:
df = pd.read_csv("./iris_data/iris.data",header = None )

x = df.iloc[:,:-1].values # İlk 4 sütunu x değişkenine atar
y , _ = pd.factorize(df.iloc[:,-1]) # Verilere ait etiketleri y değişkenine atar 

**Veriyi Standardize Etme İşlemi**

In [3]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

**Train Test Split Olarak İkiye Ayırma**

In [4]:
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size = 0.3 , random_state = 42)

**Verileri Tensor Olarak Döndürme**

In [5]:
def to_tensor(data , target):
    return torch.tensor(data , dtype = torch.float32) , torch.tensor(target , dtype = torch.long )

In [6]:
x_train , y_train = to_tensor(x_train , y_train)

In [7]:
x_test , y_test = to_tensor(x_test , y_test)

**Modelin Tanımlanması**

In [17]:
def rbf_kernel(x , centers , beta):
    return torch.exp(-beta * torch.cdist(x , centers)**2)

class RBFN(nn.Module):
    
    def __init__(self , input_dim , num_centers , output_dim ):
        super(RBFN,self).__init__()
        self.centers = nn.Parameter(torch.randn(num_centers,input_dim)) # RBF merkezlerini rastgele başlat
        self.beta = nn.Parameter( torch.ones(1) * 2.0 ) # Beta parametresi RBF'in genişliğini kontrol edecek
        self.linear = nn.Linear(num_centers , output_dim ) # Output'u tam bağlantılı katmana yönlendir 
        
    
    def forward(self , x):
        # RBF çekirdek fonk. hesapla
        phi = rbf_kernel(x , self.centers , self.beta)
        return self.linear(phi)

**Modelin Eğitimi**

In [20]:
num_centers = 10
model = RBFN(input_dim = 4 , num_centers = num_centers , output_dim = 3)

# Kayıp fonk.tanımlama ve optimizasyon
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters() , lr = 0.01)

# Model Eğitimi
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad() # Gradyanların sıfırlanması
    outputs = model(x_train) # Prediction , yani ileri yayılım
    loss = criterion(outputs,y_train) # Loss hesapla
    loss.backward() # Geri yayılım
    optimizer.step() # Ağırlıkların güncellenmesi

    if (epoch + 1) % 10 == 0:
        print(f"Epoch: {epoch+1} / {num_epochs} - Loss : {loss.item():.4f}")

Epoch: 10 / 100 - Loss : 1.1097
Epoch: 20 / 100 - Loss : 1.0748
Epoch: 30 / 100 - Loss : 1.0359
Epoch: 40 / 100 - Loss : 0.9775
Epoch: 50 / 100 - Loss : 0.8760
Epoch: 60 / 100 - Loss : 0.7273
Epoch: 70 / 100 - Loss : 0.5768
Epoch: 80 / 100 - Loss : 0.4451
Epoch: 90 / 100 - Loss : 0.3390
Epoch: 100 / 100 - Loss : 0.2631


**Modelin Test Edilmesi**

In [ ]:
with torch.no_grad():
    y_pred = model(x_test)
    accuracy = 